# 주식데이터 불러오기

In [60]:
import pandas as pd

code_df = pd.read_html('ㅎ', header=0)[0]

code_df.종목코드 = code_df.종목코드.map('{:06d}'.format)


In [61]:
#상장된 기업들목록에서 회사명과 종목코드만 가져옴
code_df = code_df[['회사명', '종목코드']]
code_df = code_df.rename(columns={'회사명':'name', '종목코드':'code'})
code_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2352 entries, 0 to 2351
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    2352 non-null   object
 1   code    2352 non-null   object
dtypes: object(2)
memory usage: 36.9+ KB


In [62]:
#회사명을 넣으면, code_df를 뒤져서 종목코드를 찾고, 네이버주식url에 덧대서 출력해주는 함수
def get_url(item_name, code_df):
    code = code_df.query("name=='{}'".format(item_name))['code'].to_string(index=False)
    code = code[1:7]
    url = 'http://finance.naver.com/item/sise_day.nhn?code={code}'.format(code=code)
    print("요청 URL = {}".format(url))
    return url

item_name = '카카오'
url = get_url(item_name, code_df)

요청 URL = http://finance.naver.com/item/sise_day.nhn?code=035720


### 날짜별 종가, 전일비, 시가, 고가, 저가, 거래량 불러오기

In [ ]:
asdfasdfasdf

In [70]:
df = pd.DataFrame()

#페이지숫자만큼 데이터 불러오기
for page in range(1,7):
    pg_url = '{url}&page={page}'.format(url=url, page=page)
    df = df.append(pd.read_html(pg_url, header=0)[0], ignore_index=True)
    
#결측값 제거
df = df.dropna()
df.head()

,날짜,종가,전일비,시가,고가,저가,거래량
1,2020.06.15,252500.0,13500.0,266000.0,271000.0,250000.0,1536566.0
2,2020.06.12,266000.0,1000.0,256500.0,268000.0,256000.0,1688975.0
3,2020.06.11,265000.0,3000.0,264000.0,268000.0,261000.0,1724551.0
4,2020.06.10,262000.0,7500.0,256500.0,263500.0,256500.0,1575452.0
5,2020.06.09,254500.0,2000.0,255500.0,259500.0,252500.0,834862.0


In [73]:
# 한글로 된 컬럼명을 영어로 바꿔줌 
df = df.rename(columns= {'날짜': 'date', '종가': 'close', '전일비': 'diff', '시가': 'open', '고가': 'high', '저가': 'low', '거래량': 'volume'}) 
# 데이터의 타입을 int형으로 바꿔줌 
df[['close', 'diff', 'open', 'high', 'low', 'volume']] = df[['close', 'diff', 'open', 'high', 'low', 'volume']].astype(int) 
# 컬럼명 'date'의 타입을 date로 바꿔줌 
df['date'] = pd.to_datetime(df['date']) 
# 일자(date)를 기준으로 오름차순 정렬 
df = df.sort_values(by=['date'], ascending=True) 
# 상위 5개 데이터 확인 
df.head()


,date,close,diff,open,high,low,volume
88,2020-03-18,141000,6500,148500,152000,140500,1107527
87,2020-03-19,134000,7000,145500,147000,127500,2157141
86,2020-03-20,149500,15500,138000,151000,134500,1765756
85,2020-03-23,143500,6000,138000,148000,137500,1569378
84,2020-03-24,154500,11000,150500,155000,147000,1271387


# 차트 그리기

In [74]:
import plotly.offline as offline
import plotly.graph_objs as go

offline.init_notebook_mode(connected=True)

trace = go.Scatter(x=df.date, y=df.close, name=item_name)
data = [trace]
# data = [celltrion] 
layout = dict( title='{}의 종가(close) Time Series'.format(item_name), xaxis=dict( rangeselector=dict( buttons=list([ dict(count=1, label='1m', step='month', stepmode='backward'), dict(count=3, label='3m', step='month', stepmode='backward'), dict(count=6, label='6m', step='month', stepmode='backward'), dict(step='all') ]) ), rangeslider=dict(), type='date' ) ) 
fig = go.Figure(data=data, layout=layout) 
offline.iplot(fig)
